# 1 Read me

Here I am going to show the code I made for the control system and also the steps I took in getting the result. For the micro-controller I had the [LAUNCHXL-F28027F](https://www.ti.com/tool/LAUNCHXL-F28027F?ds_k=LAUNCHXL-F28027F&DCM=yes&gclsrc=ds&gclsrc=ds). The boost converter was already made and was not designed for optimal closed-loop implementation, but more on that later.  

# 2 Method of implementing controls

I have 2 options if I want to implement the control program. Option 1 I design the entire thing in Simulink and then use the built-in Embedded Coder Simulink submodel and then let it do all the data conversion. This way I only need to use Simulink and then press built hardware and I am done. The second option is I make the entire code in C using the free Code Composer Studio program from Texas Instruments.

My C/C++ skill is very little, plus it can become really easy to be lost when designing and more importantly debugging problems if I decide to use this option. Simulink is a lot more user-friendly, and easier to follow and debug, but it isn't free sadly. In practice controls, engineers do complicated controls using Simulink/Matlab, thanks to my university I also get both for free so I decided to do everything in Matlab/Simulink. Maybe later I can try and do this in C/C++, but I can't promise this. I made everything using Matlab 2020 B.

# 3 Simulated model

The first step before implementing any control algorithm is to first make a simulation model of the thing u want to control. Since I am going to implement everything in Simulink/Matlab I decided to make a Simulink/Matlab simulation model. Below is a reminder of how the Boost converter looks like

<img src="Images/Implemented/Boost_circuit_with_input_cap.png" width=500> 

In my special case, I have full control over the input voltage, so I decided to use $45 V$ because that was the one the university used and I couldn't find any reason to change that. I could keep swapping all passive components to get the new desired output values, but that is a pretty dumb idea for obvious reasons. In this case, I have control over the load, but in practice, this is not always the case. So what can I control? There is only one thing left the transistors gate signal. I can control how fast it turns ON/OFF and how long it does that. And that is what I will use

## Open loop model

I am going to start with open loop control simulation model. Purely because it is the easiest to implement. 

<img src="Images/Implemented/General_open_loop_control.png" width=500> 
[Source](https://www.electronicshub.org/open-loop-system/)

In my case the input signal is the switching frequency and duty cycle and the process output is the output voltage of the boost converter.

The model i made is shown below. It can be found in [Fake_Boost]{Fake_Boost}. Yes I ignored all parasitics for my model, but I just wanted a simple implementation to see if what I think happens actually happens. I didn't want to go measure and derive a perfect simulation model that will exactly follow the measured result.

<img src="Images/Implemented/Fake_boost_open_loop_method_model.PNG" width=600> 

<img src="Images/Implemented/Fake_boost_open_loop_method_model_plot.png" width=700> 

Um the MOSFET model looks right but the IGBT overshoots a lot... Something is wrong.

For full transparency I am also adding the Maltab code I used to to upload all the parameters. It can be found in the Fake_boost_parameters Matlab code 

```Matlab
%
% @Author: Hitesh Dialani
% @Date: 09-09-2022

clc; clear; close all;

%% General Parameters
V_DC=45; % 
R_Load=400; 
C_out=25e-3;
V_f=1.5; 

%% MOSFET 
V_MOSFET_out=150;
L_MOSFET=750e-6;
D_MOSFET=(V_MOSFET_out+V_f-V_DC)/(V_MOSFET_out+V_f)*100; % Calculated duty cycle
t_sw_MOSFET=1/(80e3);

%% IGBT
V_IGBT_out=300; 
L_IGBT=1e-3;
D_IGBT=(V_IGBT_out+V_f-V_DC)/(V_IGBT_out+V_f)*100; % Calculated duty cycle
t_sw_IGBT=1/(15e3);
```

<img src="Images/Implemented/Fake_boost_open_loop_maskes_transistors.PNG" width=500> 

So my first idea is that it is the IGBT model, since everything else looks fine. So that is what I did I swapped the IGBT MASK for a MOSFET mask

<img src="Images/Implemented/Fake_boost_open_loop_method_model_plot_both_MOSFET.png" width=500> 

So I can swap the IGBT for a MOSFET to get a realistic result, but that makes no sense since when doing the actual measurements the output voltage does not overshoot, else it's overvoltage protection would kick in. So there has to be something stopping this voltage overshoot with the IGBT model? Oh wait the parasitic resistance of the inductor that i ignored... 

I now added the $R_{DC}$ to the inductor and hopefully that gives me my expected result.

<img src="Images/Implemented/Fake_boost_open_loop_method_model_solution.PNG" width=600> 

<img src="Images/Implemented/Fake_boost_open_loop_method_model_plot_solution.PNG" width=800> 

```Matlab
%
% @Author: Hitesh Dialani
% @Date: 09-09-2022

clc; clear; close all;

%% General Parameters
V_DC=45; % 
R_Load=400; 
C_out=25e-3;
V_f=1.5; 

%% MOSFET 
V_MOSFET_out=150;
L_MOSFET=750e-6;
D_MOSFET=(V_MOSFET_out+V_f-V_DC)/(V_MOSFET_out+V_f)*100; % Calculated duty cycle
t_sw_MOSFET=1/(80e3);
R_DC_MOSFET=0.3;

%% IGBT
V_IGBT_out=300; 
L_IGBT=1e-3;
D_IGBT=(V_IGBT_out+V_f-V_DC)/(V_IGBT_out+V_f)*100; % Calculated duty cycle
t_sw_IGBT=1/(15e3);
R_DC_IGBT=0.1;

```

Lesson learned don't always ignore parasitics...

## Closed  loop model

Open loop works pretty well, however as was noticed the output voltage is not exactly the desired output voltage. That is because the controller doesn't know that they are different. It only sends a fix duty cycle as was pre-calculated. So to fix this problem a closed loop system is implemented.

<img src="Images/Implemented/General_closed_loop_control.PNG" width=500> 
[Source](https://electronicscoach.com/closed-loop-control-system.html)

There are 2 methods I found current and voltage feedback, but the designed only has voltage feedback for over-voltage protection. Altium messed up and the short circuit current protection wasn't implemented correctly.. So I am only going to do voltage feedback:p 

For the feedback system I PI controller [Video](https://www.youtube.com/watch?v=UR0hOmjaHp0&t=112s), [text 1](https://www.elprocus.com/the-working-of-a-pid-controller/), [text 1](https://eng.libretexts.org/Bookshelves/Industrial_and_Systems_Engineering/Book%3A_Chemical_Process_Dynamics_and_Controls_(Woolf)/09%3A_Proportional-Integral-Derivative_(PID)_Control/9.02%3A_P%2C_I%2C_D%2C_PI%2C_PD%2C_and_PID_control)

I need to P for the gain and I need the I for the steady state error. I will not use the D, because in the end I will implement this controller on a micro controller and D in digital controllers tend to be very noisy [Source](https://controlstation.com/blog/the-d-in-pid-stands-for-do-not-use-sometimes/).

Below is the implemented control stratergy I used to get the output duty cycle

<img src="Images/Implemented/Control_stratergy.png" width=300> 

And below is the implemented boost converter with feedback in Simulink

<img src="Images/Implemented/Fake_boost_closed_loop_method_model.png" width=700> 

```Matlab
%
% @Author: Hitesh Dialani
% @Date: 09-09-2022

clc; clear; close all;
%% General Parameters
V_DC=45; % 
R_Load=400; 
C_out=25e-3;
V_f=1.5; 

%% MOSFET 
V_MOSFET_out=150;
L_MOSFET=750e-6;
D_MOSFET=(V_MOSFET_out+V_f-V_DC)/(V_MOSFET_out+V_f)*100; % Calculated duty cycle
t_sw_MOSFET=1/(80e3);
R_DC_MOSFET=0.3;

%% IGBT
V_IGBT_out=300; 
L_IGBT=1e-3;
D_IGBT=(V_IGBT_out+V_f-V_DC)/(V_IGBT_out+V_f)*100; % Calculated duty cycle
t_sw_IGBT=1/(15e3);
R_DC_IGBT=0.1;

%% Closed loop
Kp_MOSFET=0.6;
Ki_MOSFET=4.5;
 
Kp_IGBT=0.8;
Ki_IGBT=5;

```

Now to see the results

<img src="Images/Implemented/Fake_boost_closed_loop_method_model_plot.png" width=700> 

A few remarks

* Yes the IGBT model is not optimally tuned, but I got tired of tuning for an entire day
* Yes I did pure trial and error, my control theory knowledge is pretty low
* If I make Kp larger than 1 the system becomes extremely unstable, no idea why

## 3.3 Running simulation model

1) Open Fake_boost folder from Simulink folder<br>
2) Open Fake_boost_parameters.m file<br>
3) Modify Fake_boost_parameters.m if desired else just Run file <br>
4) Open Fake_boost.slx file <br>
5) Uncomment what you want to run Open or Closed loop or both <br>
6) Run Simulation <br>
7) Wait an eternity <br>
8) Results <br>

# 4 The fun part

Now for the actual fun part. The implemented controls used on the micro-controller. First step is a top level diagram.

TO DO

## Open loop no dashboard

For this method you need need 2 files

1) Openloop_voltage_control_parameters.m <br>
2) Openloop_voltage_control.slx

The full code is given in the Simulink folder, below shows only the important portion that is needed to run this model.

```Matlab
% @Author: Hitesh Dialani
% @Date: 09-09-2022
clc; clear; close all;

%% Fixed parameters
V_in=45;
V_diode_forward_drop=1.5;

%% User input
V_MOSFET_out=150;
V_IGBT_out=300;

f_sw_MOSFET=80e3;
f_sw_IGBT=15e3;

%% Calculated values 

D_MOSFET_CCM=(V_MOSFET_out+V_diode_forward_drop-V_in)/(V_MOSFET_out+V_diode_forward_drop);
D_IGBT_CCM=(V_IGBT_out+V_diode_forward_drop-V_in)/(V_IGBT_out+V_diode_forward_drop);

%% Timer stuff
Timer_period_MOSFET=round(60e6/f_sw_MOSFET);
Timer_period_IGBT=round(60e6/f_sw_IGBT);
```

Now onto the Simulink model. First off the top level model. This is shown below

<img src="Images/Implemented/Open_loop_no_dashboard_top_level.png" width=700> 

###  Input hardware block

First lets start with the input Hardware block. As the name suggest this block contains every signal signal coming from the boost PCB hardware. Below shows what is inside this block.

<img src="Images/Implemented/Input_hardware_simulink.png" width=600> 

How the systems work is as followed there is a toggle switch present on the boost PCB and depending on the state of the switch either the MOSFET or IGBT or neither PWM signal are sent.
<img src="Images/Implemented/Hardware_switch.jpeg" width=500> 

That is wat MOSFET and IGBT_Enable signals do. As for Volt_protection this is the voltage measuring circuit originally designed for over voltage protection. 

### Logic Circuit Block

<img src="Images/Implemented/Logic_circuit_Hardware_switch.png" width=600> 

### Timer block

In the end I made 2 versions of Timer blocks 1 for parameters and 1 for dashboard, so to make my code more user friendly 1 made 2 different block. For the parameters version obviously pick do the following. 
<img src="Images/Implemented/Open_loop_no_dashboard_Timer_toplevel.png" width=600>

And inside the block you get this.
<img src="Images/Implemented/Open_loop_no_dashboard_Timer_low_level.png" width=600> 

The value was calculated from this part of the code  

```Matlab

%% User input
f_sw_MOSFET=80e3;
f_sw_IGBT=15e3;

%% Timer stuff
Timer_period_MOSFET=round(60e6/f_sw_MOSFET);
Timer_period_IGBT=round(60e6/f_sw_IGBT);
```

### Duty block

### Output block

Finally the output. Below shows all the signals that are sent from the micro-controller. First are the LEDs. There are 3 LEDs that are controlled and are connected to a RGB LED. What the colors mean are as followed. If the GREEN LED is shown then the micro-controller is on OFF state. None of the PWM signal for the transistors are enabled and there are now errors. If the BLUE LED is illuminated than 1 or both of the transistors PWM signals are sent. Finally the RED LED. If the that illuminate than the over-voltage was activated. 

Next are the ePWM block set. The ePWM block set receives the switching time, duty cycle and PWM enable signal. All 3 signals come from other block.

<img src="Images/Implemented/Output_simulink.png" width=600> 

## Open loop with dashboard

## Closed loop with dashboard